IMPORTING LIBRARIES

In [1]:
import pandas as pd
import requests
import lxml
import csv
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


/tmp/ipykernel_57780/2215096945.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


SETTING HEADLESS CHROME

In [2]:
options=Options()
options.add_argument('--headless')

CREATING A FUNCTION TO GET LINKS TO THE LAST 10 SEASONS TO EXTRACT OUR DATA FROM

In [3]:
baseUrl="https://www.flashfootball.com/england/premier-league-2022-2023/archive/"

In [4]:
def getSeasonLinks(baseUrl):
    links=[]
    archive=webdriver.Chrome(options=options)
    archive.get(baseUrl)
    WebDriverWait(archive,10).until(EC.element_to_be_clickable((By.ID,"onetrust-accept-btn-handler"))).click()
    #archive.find_element(By.ID,"onetrust-accept-btn-handler").click()
    section=archive.find_element(By.ID,"tournament-page-archiv")
    seasons=section.find_elements(By.CLASS_NAME,"archive__row")
    for season in seasons[1:10]:
        link=season.find_element(By.CLASS_NAME,"archive__season").find_element(By.TAG_NAME,'a').get_attribute('href')
        links.append(link)
    return links

FN TO GET THE PAGE DATA OF EACH SEASON LINK

In [5]:
def getPage(link):
    driver=webdriver.Chrome(options=options)
    driver.get(link)
    driver.save_screenshot("page.png")
    #driver.implicitly_wait(5)
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,"onetrust-accept-btn-handler"))).click()
   # driver.find_element(By.ID,"onetrust-accept-btn-handler").click()
    return driver

CREATING A SOUP OF THE WEBELEMENT PAGE SOURCE TO EXTRACT 1ST LEVEL DATA

In [7]:
def getSoupOfGames(driver):
    soup=BeautifulSoup(driver.page_source,'html.parser')
    live=soup.find('div',id="live-table")
    event=live.find('div',class_="leagues--static event--leagues results")
    sport=event.find('div',class_="sportName soccer")
    games=sport.find_all('div',{'class':
                                [
                                    'event__match event__match--static event__match--twoLine',
                                    'event__match event__match--static event__match--last event__match--twoLine'
                                ]})
    return games

LOOPING THROUGH EACH SEASON LINK FETCHING BASIC DATA STATS

In [8]:
links=getSeasonLinks(baseUrl)

In [9]:
links

['https://www.flashfootball.com/england/premier-league-2022-2023/',
 'https://www.flashfootball.com/england/premier-league-2021-2022/',
 'https://www.flashfootball.com/england/premier-league-2020-2021/',
 'https://www.flashfootball.com/england/premier-league-2019-2020/',
 'https://www.flashfootball.com/england/premier-league-2018-2019/',
 'https://www.flashfootball.com/england/premier-league-2017-2018/',
 'https://www.flashfootball.com/england/premier-league-2016-2017/',
 'https://www.flashfootball.com/england/premier-league-2015-2016/',
 'https://www.flashfootball.com/england/premier-league-2014-2015/']

In [19]:
for link in links[:1]:
    #GETTING OUR PAGES
    driver=getPage(link)
    #CLICKING ON "SHOW MORE MATCHES" TO DISPLAY ALL THE MATCHES IN THE PAGE B4 CREATING THE SOUP
    x=0
    while x<4:
        try:
            #driver.implicitly_wait(5)
            element=WebDriverWait(driver,20).until(
            EC.element_to_be_clickable((By.CLASS_NAME,"event__more--static"))
            )
            driver.execute_script("arguments[0].scrollIntoView();", element)
            element.click()
            #driver.save_screenshot("more.png")
            x+=1 
        except ElementClickInterceptedException:
            driver.execute_script("arguments[0].click()",element)
            x=x+1
        #CREATING SOUP OBJECT
        games=getSoupOfGames(driver)
        #GETTING CURRENT SEASON
        obj=re.compile(r'\d{4}\-\d{4}')
        season=obj.findall(link)
        #GETTIN INDIVIDUAL GAME STATS FOR EACH MATCH
        for game in games[:50]:
            home=game.find('div',{'class':["event__participant event__participant--home fontExtraBold","event__participant event__participant--home"]}).text
            homegoal=game.find('div',class_='event__score event__score--home').text
            away=game.find('div',{'class':["event__participant event__participant--away fontExtraBold","event__participant event__participant--away"]}).text
            awaygoal=game.find('div',class_='event__score event__score--away').text
            awaygoals.append(awaygoal)
            id=game.get('id')
            id=id[4:]
            link="https://www.flashfootball.com/match/%s/#/match-summary/match-summary"%id
            driver1=webdriver.Chrome(options=options)
            driver1.get(link)
            driver1.implicitly_wait(5)
            WebDriverWait(driver1,20).until(EC.element_to_be_clickable((By.ID,"onetrust-accept-btn-handler"))).click()
            #driver1.find_element(By.ID,"onetrust-accept-btn-handler").click()
            driver1.implicitly_wait(5)
            summary=driver1.find_element(By.CSS_SELECTOR,"#fullPageDetail > div.filterOver.filterOver--indent > div > a:nth-child(2)")#.click()
            driver1.execute_script("arguments[0].scrollIntoView()",summary)
            driver1.execute_script("arguments[0].click()",summary)
            driver1.implicitly_wait(7)
            driver1.find_element(By.CSS_SELECTOR,"#fullPageDetail > div.subFilterOver.subFilterOver--indent.subFilterOver--radius > div > a.active").click()
            sp=BeautifulSoup(driver1.page_source,'html.parser')
            stat=sp.select_one("#fullPageDetail > div:nth-child(8)")
            diffstats=stat.find_all('div',class_='_row_rz3ch_9')
            cth=[];cta=[];hmst=[];awst=[]
            for row in diffstats:
                main=row.find('div',class_="_category_rz3ch_16")
                category=main.find('div',class_="_category_1x9y9_5").find('strong').text#_category_1x9y9_5
                homestat=main.find('div',class_="_value_dmww2_5 _homeValue_dmww2_10").find('strong').text
                awaystat=main.find('div',class_="_value_dmww2_5 _awayValue_dmww2_14").find('strong').text
                categoryaway=str(category)+"away"
                categoryhome=str(category)+"home"
                cth.append(categoryhome)
                cta.append(categoryaway)
                hmst.append(homestat)
                awst.append(awaystat)
            categories=["season","hometeam","homegoals","awayteam","awaygoals"]+cth+cta
            rowmain=[season,home,homegoal,away,awaygoal]+hmst+awst
            dic={}
            for x in range(len(categories)):
                dic[categories[x]]=rowmain[x]
            with open ('matches2.csv','a',newline='')as new:
                writerr=csv.DictWriter(new,fieldnames=categories)
                writerr.writeheader()
                writerr.writerow(dic)
                    